In [ ]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install fake_useragent

from fake_useragent import UserAgent
# selenium 불러오기 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
base_url = "https://www.tbc.net.tw/Epg/"

In [40]:
#@title Get the selenium initial_chrome driver
#base_url = "https://www.tbc.net.tw/Epg/"
 
# chrome창(웹드라이버) 열기 
chrome_options = webdriver.ChromeOptions()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
chrome_options.add_argument(f'user-agent={userAgent}')
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("lang=ko_KR") # 한국어!
# UserAgent값을 바꿔줍시다!
#chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66)Safari/537.36")

 

 


driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
# TBC 접속하기 
driver.implicitly_wait(3)
driver.get(base_url)

driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5]}})")
# lanuages 속성을 업데이트해주기
driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko']}})")
driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'NVIDIA Corporation'} if (parameter === 37446) {return 'NVIDIA GeForce GTX 980 Ti OpenGL Engine';}return getParameter(parameter);};")
wait = WebDriverWait(driver, 10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


NoSuchElementException: ignored

In [ ]:
#@title Make sure you are not really headleas

In [ ]:
print(driver.execute_script("if(navigator.plugins.length === 0){console.log('Headless 크롬이 아닐까?');};"))

None


In [ ]:
#@title Update the date of driver funtion
#https://sacko.tistory.com/15
#https://dzone.com/articles/perform-actions-using-javascript-in-python-seleniu
# inputting values in the text box with help of Javascript executor
# javaScript = "document.getElementsByClassName('gsc-input')[0].value = 'TestNG' "
# driver.execute_script(javaScript)
"""old code:
mySelect = Select(driver.find_element_by_id("SearchParameter_Hour"))
mySelect.select_by_visible_text("00:00") # mySelect.select_by_value("Value") # mySelect.select_by_index(0)
"""
#     $(document).ready(function () {
#         if(GetBeforeHeight>0){
#             $(document).scrollTop(GetBeforeHeight);
#             $.unblockUI();
#         }
                
#                 $('.bt_left').click(function () {
#                     var scrollLeft = $('.srl')[0].scrollLeft;
#                     if (scrollLeft > 0) {
#                         $('.srl').scrollLeft(0);
#                         return false;
#                     }
#                     $('#SearchParameter_StartDate').val('2021/02/27');
#                     $('#SearchParameter_Hour').val('8');
#                     $('#epgForm').append($('<input>').attr({ type: 'hidden', id: 'TargetHeight', name: 'TargetHeight', value: $(document).scrollTop() }));
#                     $('#epgForm').submit();
#                     showLoding();
#                     return false;
#                 });

# <a href="#" onclick="$('#SearchParameter_EpgChannelKeyword').val('');$('#epgForm').submit();showLoding();return false;" class="icon_cancel"></a>




'old code:\nmySelect = Select(driver.find_element_by_id("SearchParameter_Hour"))\nmySelect.select_by_visible_text("00:00") # mySelect.select_by_value("Value") # mySelect.select_by_index(0)\n'

In [ ]:
def update_date(driver,date:str)->str: #ex)2021/02/28
  driver = driver
  driver.execute_script("($'#SearchParameter_Hour').val('0');")
  driver.execute_script("($'#SearchParameter_StartDate').val({a_date});".format(a_date = date))
  #element = driver.find_element_by_id("SearchParameter_StartDate")
  update_btn = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
  result = driver.execute_script("arguments[0].click();", update_btn)
  print("Succesfuly updated")
  return (result)

In [ ]:
#@markdown Check if the drive can be succefully updated
#@title Sample cawling exist[Click twice here to check]
#get a link
def sample_info(driver):
  driver = driver
  alink = "/html/body/div[1]/div[2]/div[4]/div[5]/div/ul[66]/li[1]/a"
  pop = driver.find_element_by_xpath(alink)
  #Go!
  webdriver.ActionChains(driver).move_to_element(pop).click().perform()
  info = [driver.find_element_by_xpath('//*[@id="prg_name"]').text
          ,driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
          ,driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
          ,driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")
          ,driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()]
  #print(Info)
  return (info)

def current_tmLine(driver):
  driver = driver
  TM_list = driver.find_elements_by_xpath('//*[@id="_epg_time_list"]/li')
  TimeL = []
  for tm in TM_list:
    TimeL.append(tm.get_attribute("datetime"))
  return (TimeL)

In [ ]:
print("Original driver info:{} \n tmLine:{} \n".format(sample_info(driver),current_tmLine(driver)[0]))
result = update_date(driver,'2021/03/03') 
print("updated_driver info:{} \n tmLine:{}".format(result.sample_Info(result),result.current_tmLine(result)[0]))

Original driver info:['極速快感', ['類別：動作、西洋電影      導演：史考特沃爾'], '電影場景忠實呈現遊戲中的緊張與刺激：警匪街頭驚險追逐畫面、超跑競速、甩尾、跳躍；體驗競速的淋漓盡致與神經緊繃，無縫接軌不可思議的極限與快感。', '2021/02/27', []] 
 tmLine:2021/02/27 17:00 



JavascriptException: ignored

In [ ]:
# inputting values in the text box with help of Javascript executor
javaScript = "document.getElementsByClassName('ui-datepicker-trigger')[0].value = 'TestNG' "
driver.execute_script(javaScript)

In [ ]:
#@title Make a funtion changing time&date

In [ ]:
def init_date(date:str) -> str:
  element = driver.find_element_by_id("SearchParameter_StartDate")
  element.send_keys("abcdefg@email.com")
  return print("The date of Epg is moved to the date{}".format(a_date : "date"))


def get_main_time():
  mySelect = Select(driver.find_element_by_id("SearchParameter_Hour"))
  mySelect.select_by_visible_text("00:00") 

def Next_time_line():
  first_date = str_date
  last_date = end_date
  element = driver.find_element_by_name("emailOrPhone")

def get_24hr_info():
  mySelect = Select(driver.find_element_by_id("SearchParameter_Hour"))
  mySelect.select_by_visible_text("00:00") # mySelect.select_by_value("Value") # mySelect.select_by_index(0)


In [ ]:
#@title *Experiment* ----------  Find a way to scrap a week of epg info all at once

In [ ]:
#@markdown Check the prejudged time line on the current page
TM_list = driver.find_elements_by_xpath('//*[@id="_epg_time_list"]/li')
for tm in TM_list:
  print(tm.get_attribute("datetime"))

2021/02/27 12:00
2021/02/27 12:30
2021/02/27 13:00
2021/02/27 13:30
2021/02/27 14:00
2021/02/27 14:30
2021/02/27 15:00
2021/02/27 15:30
2021/02/27 16:00
2021/02/27 16:30
2021/02/27 17:00
2021/02/27 17:30


In [ ]:
#@markdown show me the all choosable hours on the selection bar [used JS command]
#https://dzone.com/articles/perform-actions-using-javascript-in-python-seleniu
javaScript = "document.getElementById('SearchParameter_Hour').click();"
driver.execute_script(javaScript)
print (driver.execute_script('return document.getElementById("SearchParameter_Hour").innerText'))

00:00
01:00
02:00
03:00
04:00
05:00
06:00
07:00
08:00
09:00
10:00
11:00
12:00
13:00
14:00
15:00
16:00
17:00
18:00
19:00
20:00
21:00
22:00
23:00


In [ ]:
#@markdown Can I change the time line, starting from  00:00?<br> 
#@markdown Let's try it

In [ ]:
#@markdown Solution :  Use "select function" that implicitly clicks the  option element of the dropdown menu
#https://sqa.stackexchange.com/questions/12029/how-do-i-work-with-dropdowns-in-selenium-webdriver
#javaScript = "document.getElementById('SearchParameter_Hour').click();"
#driver.execute_script(javaScript)
"""1. driver.selectByVisibleText("Text");
 2. driver.selectByIndex(1);
 3. driver.selectByValue("prog");"""
mySelect = Select(driver.find_element_by_id("SearchParameter_Hour"))
mySelect.select_by_visible_text("00:00") # mySelect.select_by_value("Value") # mySelect.select_by_index(0)

In [ ]:
#@markdown Then,let's see if it works
TM_list = driver.find_elements_by_xpath('//*[@id="_epg_time_list"]/li')
for tm in TM_list:
  print(tm.get_attribute("datetime"))

2021/02/25 00:00
2021/02/25 00:30
2021/02/25 01:00
2021/02/25 01:30
2021/02/25 02:00
2021/02/25 02:30
2021/02/25 03:00
2021/02/25 03:30
2021/02/25 04:00
2021/02/25 04:30
2021/02/25 05:00
2021/02/25 05:30


In [ ]:
#@markdown It Works!

In [ ]:
#@title *Experiment done* ----------

In [ ]:
#@title Let's get it down!
#https://www.javatpoint.com/python-exception-handling

In [ ]:
"""Wait and Check the len of EPG and the group of corresponding CNT """
try:  
  epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
  cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
  if (len(epg) != len(cnt)):  
# we can pass the message in the raise statement  
    raise ValueError("the number of channels and contents lists don't match!!") 
  else:
    print('the num of channels and the num of lists of contents on the table match! Keep crawling!') 
    print("the each num of channel and lists of contents is {}".format(len(epg)))
except ValueError as e:  
     print(e)  

the num of channels and the num of lists of contents on the table match! Keep crawling!
the each num of channel and lists of contents is 225


In [ ]:
#@title Create CLOSE btn!!
close_btn = '//*[@id="epg_popup"]/div/div[1]/div[3]/a'
close_btn_ele = driver.find_element_by_xpath(close_btn)
close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_ele).click()
#close_x.perform()

In [ ]:
driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()

[]

In [ ]:
#@markdown Next, we get the epg info.

In [ ]:
epg_info = []
for i in range(len(epg)):
  
  tv_name = epg[i].text.split('\n') #get the channel name
  a_ctn_list = WebDriverWait(cnt[i], 10).until(
      EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) # match to the corresponding CH
  # equals --> a_ctn_list = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)) # match to the corresponding CH#
  ctn_info = []
  for j in range(len(a_ctn_list)): #get the each info from the correspoing lists of contents
    a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click()
    a_ctn.perform()
    try: # if one of info is empty, we just leave it as a blank
      ctn_info.append([driver.find_element_by_xpath('//*[@id="prg_name"]').text] +
                  wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_moreinfo"]/p'))).text.split('\n') +
                  [driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")] +
                  [driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")] +  --> 다 wait으로 바꾸기
                  driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
                  )
    
    except NoSuchElementException:
      # Just append a None or ""
      ctn_info.append(['']+['']+['']+['']+[''])
    close_pop.perform()  
    epg_info.append({'channel': tv_name,
                      'prog': list(zip(*ctn_info))[0],
                      'genre': list(zip(*ctn_info))[1],
                      'desc': list(zip(*ctn_info))[2],
                      'date': list(zip(*ctn_info))[3],
                      'time': list(zip(*ctn_info))[4]
                     })


In [ ]:
#@title Save the Genres variables
#save variables
import pickle
with open('ctn_info','wb') as f:
  pickle.dump(ctn_info, f)
with open('epg_info','wb') as f:
  pickle.dump(epg_info, f)

In [ ]:
ctn_info

[['笑吧東海:11',
  '類別：亞洲連續劇      導演：金明旭;毛完日',
  '講述四十幾年前被收養到美國的精神年齡只有9歲的汝茵（都知嫄飾）和兒子東海回到韓國，尋找兒子的親生父親並在這個過程中找到愛情和家人故事。',
  '2021/02/25',
  '16:00~18:00']]

[{'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25',),
  'desc': ('系統資訊',),
  'genre': ('類別：知性（知識）',),
  'prog': ('系統資訊',),
  'time': ('17:00~17:01',)},
 {'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25', '2021/02/25'),
  'desc': ('系統資訊', '數位小百科'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）'),
  'prog': ('系統資訊', '數位小百科'),
  'time': ('17:00~17:01', '17:01~17:44')},
 {'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25', '2021/02/25', '2021/02/25'),
  'desc': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'prog': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳'),
  'time': ('17:00~17:01', '17:01~17:44', '17:44~18:00')},
 {'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25', '2021/02/25', '2021/02/25', '2021/02/25'),
  'desc': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳', '系統資訊'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'prog': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳', '系統資訊'),
  'time': ('17:00~17:01', '17:01~17:44', '17:44~18:00', '18:00~18:01')},
 {'ch

In [ ]:

for i in range(len(epg)):
  lists = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul')
  ch_name = epg[i].text.split('\n')
  epg_info = []
  for j in range(len(lists)):
    a_list = lists[j].find_elements_by_xpath('//*[@id="content"]/ul[1]/li/a') 
    #variable for following prog_info
    prog_info = []
    for k in range(len(a_list)):
    #Click a link from a list of programs
      ele = webdriver.ActionChains(driver).move_to_element(a_list[k]).click()
      ele.perform()
      #Get the info from the link
      try:
        #Insert the scraping action here
        name = driver.find_element_by_xpath('//*[@id="prg_name"]').text
        genre = driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
        desc = driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
        date = driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")
        time = driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
        prog_info.append([name] + genre + [desc] + [date] + time) 
      except NoSuchElementException:
        # Just append a None or ""
        prog_info.append(['']+['']+['']+['']+[''])
      close_pop.perform()
    epg_info.append({'channel': ch_name,
                      'prog': list(zip(*prog_info))[0],
                      'genre': list(zip(*prog_info))[1],
                      'desc': list(zip(*prog_info))[2],
                      'date': list(zip(*prog_info))[3],
                      'time': list(zip(*prog_info))[4]})

In [ ]:
'''Check if it changes after clicking the other time btn'''
#default location for time setting BTN
search_tm_Xpath = '//*[@id="SearchParameter_Hour"]/option'
search_tm_btn = driver.find_elements_by_xpath(search_tm_Xpath)
selected_tm = webdriver.ActionChains(driver).move_to_element(search_tm_btn[0]).click()
selected_tm.perform()
# selected_tm = webdriver.ActionChains(driver).move_to_element(search_tm_btn).click()
# selected_tm.perform()
# #check the result first
# print(//*[@id="_epg_time_list"]/li[1])
print(search_tm_btn)



In [ ]:
# i.e #text

In [ ]:
#https://dzone.com/articles/perform-actions-using-javascript-in-python-seleniu
#getElementsByClassName
#getElementById
#getElementsByName
'''TBC time line btn : document.querySelector("#SearchParameter_Hour")'''
javaScript = "document.getElementsByName('username')[0].click();"
driver.execute_script(javaScript)

userName = driver.find_element_by_xpath("//button[@name='username']")
driver.execute_script("arguments[0].click();", userName)
driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//button[@name='username']"))


print (driver.execute_script('return document.getElementById("fsr").innerText'))


In [ ]:
.//*[contains(@onclick, '231')]

In [ ]:
#@markdown Click  "00:00" btn to set as the starintg time for crawling

In [ ]:
#default location for time setting BTN
search_tm_Xpath = '//*[@id="SearchParameter_Hour"]'
search_tm_btn = driver.find_element_by_xpath(search_tm_Xpath)
selected_tm = webdriver.ActionChains(driver).move_to_element(search_tm_btn).click()
selected_tm.perform()
#check the result first
print(//*[@id="_epg_time_list"]/li[1])


In [ ]:
#Select the value for the defalut hour 00:00

In [ ]:
#@markdown Check if there are changes on outputs

In [ ]:
#_epg_time_list > li:nth-child(1)

In [ ]:
#@title Select the value for the defalut hour 00:00

In [ ]:
#SearchParameter_Hour > option:nth-child(1)

In [ ]:
#@title Only save the contents that exist

In [ ]:
wait = WebDriverWait(driver, 10)
program = wait.until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/div/div[2]/div[4]/div[5]/div/ul/li/a")))

In [ ]:
#@title Create go_next_page btn & close_pop_up_btn to keep cralwing

In [ ]:
next_page_btn = '//*[@id="epgtop"]/div[2]/div/div[2]/a'
close_btn = '//*[@id="epg_popup"]/div/div[1]/div[3]/a'
 
next_page_click = driver.find_element_by_xpath(next_page_btn)
close_btn_click = driver.find_element_by_xpath(close_btn)
 
go_next_page = webdriver.ActionChains(driver).move_to_element(next_page_click).click()
close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_click).click()

In [ ]:
#@title Sameple cawling exist[Click twice here to check]
#get a link
alink = "/html/body/div[1]/div[2]/div[4]/div[5]/div/ul[66]/li[1]/a"
pop = driver.find_element_by_xpath(alink)
#Go!
webdriver.ActionChains(driver).move_to_element(pop).click().perform()
Info = [driver.find_element_by_xpath('//*[@id="prg_name"]').text
        ,driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
        ,driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
        ,driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")
        ,driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()]
print(Info)

['變形金剛-復仇之戰', ['類別：西洋電影、科幻      導演：麥可·貝'], '片中交代了變形金剛與人類發展的歷史，讓變形金剛的角色變化更多樣，還增添埃及冒險考古色彩。變形金剛從遠古時代一路發展到現代，足跡遍布全世界，博派大英雄柯博文現身上海的畫面氣勢十足，加上狂派大壞蛋密卡登復活，更讓博狂兩派的戰爭橫跨陸、海、空，場面壯觀。', '2021/02/24', ['15:50~19:00']]


In [ ]:
#@title Start crawling exists- 1st method

In [ ]:
def match():
  epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
  cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[1]/li/a')))
  return len(epg)==len(cnt)

In [ ]:
match()

False

In [ ]:
try:
  
  if match()==True:
    raise ValueError("The length of two data match")
except Exception as ValueError:
  print("The number of channels and lists of contents don't match!!")

In [ ]:
for idx, val in enumerate(driver.find_elements_by_xpath('//*[@id="epg_popup"]/div/div[1]/div[2]/span')):
  print(idx, val)
    # Do something using idx and val

0 <selenium.webdriver.remote.webelement.WebElement (session="3d10923b36dc300125b63303568cde56", element="e74a677c-3093-468a-805f-07fe7634eb62")>
1 <selenium.webdriver.remote.webelement.WebElement (session="3d10923b36dc300125b63303568cde56", element="7c884739-17c3-479e-ad03-4ade247c3190")>


In [ ]:
val

<selenium.webdriver.remote.webelement.WebElement (session="3d10923b36dc300125b63303568cde56", element="7c884739-17c3-479e-ad03-4ade247c3190")>

In [ ]:
driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()

[]

In [ ]:
time = ( [i.split()] for i in driver.find_elements_by_xpath('//*[@id="epg_popup"]/div/div[1]/div[2]/span'))

In [ ]:
time

<generator object <genexpr> at 0x7f55f91df150>

In [ ]:
>>> a = [2,3,4,5,6,7,8,9,0]
... xyz = [0,12,4,6,242,7,9]
... for x in xyz:
...     if x in a:
...         print(x)
0,4,6,7,9
And I know I can use a list comprehension to combine these when the statements are simple, such as:

print([x for x in xyz if x in a])

In [ ]:
"""Wait and Check the len of EPG and the group of corresponding CNT """
try:
   epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
   cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[1]/li/a')))
   len(epg)==len(cnt)
except:
  print("the number of channels and lists of contents don't match!!")
 
while
      epg_info = []
      for i in range(len(epg)):
        lists = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul')
        ch_name = epg[i].text.split('\n')
        for j in range(len(lists)):
          a_list = lists[j].find_elements_by_xpath('//*[@id="content"]/ul[1]/li/a')
          #variable for following prog_info
          prog_info = []
          for k in range(len(a_list)):
          #Click a link from a list of programs
            webdriver.ActionChains(driver).move_to_element(a_list[k]).click().perform()
            #Get the info from the link
            try:
              #Insert the scraping action here
              name = driver.find_element_by_xpath('//*[@id="prg_name"]').text
              genre = driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
              desc = driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
              time = driver.find_element_by_xpath('//*[@id="epg_popup"]/div/div[1]/div[2]/span').text.split()
              prog_info.append([name] + genre + [desc] + [date] + time )
            except NoSuchElementException:
              # Just append a None or ""
              prog_info.append(['']+['']+[''])
            #close the pop-up window
            try:
              assert(close_pop.perform())
            except:
              break
 
        epg_info.append()

In [ ]:
a_list[0]

<selenium.webdriver.remote.webelement.WebElement (session="3d10923b36dc300125b63303568cde56", element="47999d33-8f3f-4bd4-aa52-e5da02f856f1")>

In [ ]:
len(epg_info)

338

In [ ]:
epg_info

[{'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25',),
  'desc': ('系統資訊',),
  'genre': ('類別：知性（知識）',),
  'prog': ('系統資訊',),
  'time': ('17:00~17:01',)},
 {'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25', '2021/02/25'),
  'desc': ('系統資訊', '數位小百科'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）'),
  'prog': ('系統資訊', '數位小百科'),
  'time': ('17:00~17:01', '17:01~17:44')},
 {'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25', '2021/02/25', '2021/02/25'),
  'desc': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'prog': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳'),
  'time': ('17:00~17:01', '17:01~17:44', '17:44~18:00')},
 {'channel': ['2', '南桃園節目總表'],
  'date': ('2021/02/25', '2021/02/25', '2021/02/25', '2021/02/25'),
  'desc': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳', '系統資訊'),
  'genre': ('類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）', '類別：知性（知識）'),
  'prog': ('系統資訊', '數位小百科', 'TBC形象宣傳&頻道宣傳', '系統資訊'),
  'time': ('17:00~17:01', '17:01~17:44', '17:44~18:00', '18:00~18:01')},
 {'ch

In [ ]:
ele = webdriver.ActionChains(driver).move_to_element(a_list[0]).click()
ele.perform()

In [ ]:
webdriver.ActionChains(driver).move_to_element(a_list[1]).click().perform()

In [ ]:
# """Wait and Check the len of EPG and the group of corresponding CNT """
# try:
#    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
#    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[1]/li/a')))
#    len(epg)==len(cnt)
# except:
#   print("the number of channels and lists of contents don't match!!")
 
 
 
for i in range(len(epg)):
  lists = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul')
  ch_name = epg[i].text.split('\n')
  epg_info = []
  for j in range(len(lists)):
    a_list = lists[j].find_elements_by_xpath('//*[@id="content"]/ul[1]/li/a') <<---format을 사용해요 그렇게 해서 다음 항으로 넘어가게 하세요
    #variable for following prog_info
    prog_info = []
    for k in range(len(a_list)):
    #Click a link from a list of programs
      ele = webdriver.ActionChains(driver).move_to_element(a_list[k]).click()
      ele.perform()
      #Get the info from the link
      try:
        #Insert the scraping action here
        name = driver.find_element_by_xpath('//*[@id="prg_name"]').text
        genre = driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
        desc = driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
        date = driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")
        time = driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
        prog_info.append([name] + genre + [desc] + [date] + time) 
      except NoSuchElementException:
        # Just append a None or ""
        prog_info.append(['']+['']+['']+['']+[''])
      close_pop.perform()
    epg_info.append({'channel': ch_name,
                      'prog': list(zip(*prog_info))[0],
                      'genre': list(zip(*prog_info))[1],
                      'desc': list(zip(*prog_info))[2],
                      'date': list(zip(*prog_info))[3],
                      'time': list(zip(*prog_info))[4]})

StaleElementReferenceException: ignored

In [ ]:
#@title Start crawling exists- 2nd method

In [ ]:
size = len(driver.find_elements_by_xpath('//*[@id="epg"]/div[4]/div/ul/li'))
for i in range(0, size):

'系統資訊'

In [ ]:
#@title Check the data

In [ ]:
prog_genre

[['系統資訊', '類別：知性（知識）', '系統資訊']]

In [ ]:
#@title Start crawling - 2st method
element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'alink2')))
sign_in.click()

In [ ]:
element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'alink2')))

In [ ]:
webdriver.ActionChains(driver).move_to_element(program[i]).click().perform()

In [ ]:
이거!!wait.until(EC.element_to_be_clickable((webdriver.ActionChains(driver).move_to_element(program[i])))

In [ ]:
이것도 --> if elem.is_displayed() and elem.is_enabled():

In [ ]:
prog_genre = []
for i in range(len(program[:1])):
  #Click prog window
  wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'alink2')))
  click_elem_btn = webdriver.ActionChains(driver).move_to_element(program[i]).click()
  click_elem_btn.perform()
  # Get the 3 infos of the prog
  try:
    # Insert your scraping action here
    prog_nam = driver.find_element_by_xpath('//*[@id="prg_name"]').text
    genr = driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
    desc = driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
    prog_genre.append([prog_nam] + genr + [desc])
  except NoSuchElementException:
    # take a sleep and append a None or ""
    time.sleep(2)
    prog_genre.append(['']+['']+[''])
  #close the window
  close_pop.perform()

In [ ]:
prog_genre = []
 
for i in range(len(program)):
  #Click prog window
  cl_ele = webdriver.ActionChains(driver).move_to_element(program[i]).click()
  cl_ele.perform()
  # Get the 3 infos of the prog
  try:
    # Insert your scraping action here
    prog_nam = driver.find_element_by_xpath('//*[@id="prg_name"]').text
    genr = driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n')
    desc = driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")
    Prog_genr.append([prog_nam] + genr + [desc])
  except NoSuchElementException:
    # Just append a None or ""
    Prog_genr.append(['']+['']+[''])
  #close the window
  close_x.perform()
 
# elementsFromPoint에러가 있는 코드임 고쳐야함 모두 1476인데 1353만 받아냈음

In [ ]:
#@title Save the Genres variables
#save variables
import pickle
with open('Prog_genr','wb') as f:
  pickle.dump(Prog_genr, f)

In [ ]:
!ls

drive  Prog_genr  sample_data


In [ ]:
#@title load the Genres variables
# Open variables
with open('Prog_genr', 'rb') as f:
    Prog_genr = pickle.load(f)

In [ ]:
Prog_genr

[['系統資訊', '類別：知性（知識）', '系統資訊'],
 ['數位小百科', '類別：知性（知識）', '數位小百科'],
 ['頻道宣傳', '類別：知性（知識）', '頻道宣傳'],
 ['系統資訊', '類別：知性（知識）', '系統資訊'],
 ['數位小百科', '類別：知性（知識）', '數位小百科'],
 ['TBC形象宣傳&頻道宣傳', '類別：知性（知識）', 'TBC形象宣傳&頻道宣傳'],
 ['系統資訊', '類別：知性（知識）', '系統資訊'],
 ['數位小百科', '類別：知性（知識）', '數位小百科'],
 ['頻道宣傳', '類別：知性（知識）', '頻道宣傳'],
 ['系統資訊', '類別：知性（知識）', '系統資訊'],
 ['數位小百科', '類別：知性（知識）', '數位小百科'],
 ['頻道宣傳', '類別：知性（知識）', '頻道宣傳'],
 ['系統資訊', '類別：知性（知識）', '系統資訊'],
 ['數位小百科', '類別：知性（知識）', '數位小百科'],
 ['頻道宣傳', '類別：知性（知識）', '頻道宣傳'],
 ['系統資訊', '類別：知性（知識）', '系統資訊'],
 ['數位小百科', '類別：知性（知識）', '數位小百科'],
 ['TBC形象宣傳&頻道宣傳', '類別：知性（知識）', 'TBC形象宣傳&頻道宣傳'],
 ['公益廣告', '類別：綜藝/娛樂', '公益廣告'],
 ['公益廣告', '類別：綜藝/娛樂', '公益廣告'],
 ['桃園新聞', '類別：新聞', '桃園新聞'],
 ['公益廣告', '類別：綜藝/娛樂', '公益廣告'],
 ['公益廣告', '類別：綜藝/娛樂', '公益廣告'],
 ['社區桃花源',
  '類別：知性（知識）',
  '社區桃花源節目，由外景主持人帶領觀眾朋友，輕鬆無負擔的收視環境，從社區的美麗景觀到史蹟、人文、特色，社區關懷據點及社區營造成果為重點，深入了解每一個社區的獨特魅力，居民們對社區所投注的情感，雖然每一個社區的人文風情都不盡相同；但相同的是，社區居民凝聚起來，那種對社區情感。所以節目的屬性就是生活化與在地化，主角就是社區居民實境紀錄，展現樸實不做作的影像紀錄，從平實輕鬆的節目內容中，引導收視觀眾，更

In [ ]:
#@title Create Genres dataframe and count it
#https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
# create datagrame for Genre
df = pd.DataFrame(Prog_genr)
df.columns =['prog_name','genre','desc']
print(len(df.index))

508


In [ ]:
#@title Check the collected dataframe
df

,prog_name,genre,desc
0,系統資訊,類別：知性（知識）,系統資訊
1,數位小百科,類別：知性（知識）,數位小百科
2,頻道宣傳,類別：知性（知識）,頻道宣傳
3,系統資訊,類別：知性（知識）,系統資訊
4,數位小百科,類別：知性（知識）,數位小百科
...,...,...,...
503,魔導少年,類別：卡通/兒童、亞洲電影,故事講述魔導士公會·妖精尾巴的成員納茲、哈比、露西和其他夥伴，如何以勇氣、友情和羈絆度過重重...
504,魔導少年,類別：卡通/兒童、亞洲電影,故事講述魔導士公會·妖精尾巴的成員納茲、哈比、露西和其他夥伴，如何以勇氣、友情和羈絆度過重重...
505,復仇者聯盟-終局之戰,類別：西洋電影、科幻、動作 導演：安東尼羅素;喬羅素,復仇者聯盟系列最終章！灰飛煙毀後的世界該如何走下去，是否還有逆轉結局的機會？一切都將在此揭曉。
506,老酒館:1,類別：亞洲連續劇,1928年到1949年的東北，「闖關東」來東北的小人物陳懷海，在東北深山老林里挖參討生活，卻...


In [ ]:
#@title Randomly select rows and check it
#randomly select rows
#https://www.geeksforgeeks.org/how-to-randomly-select-rows-from-pandas-dataframe/
df.sample(n =3)

,genre,prog_name
23,類別：綜藝/娛樂,公益時段/生活資訊
0,類別：知性（知識）,系統資訊
18,類別：新聞,桃園新聞


In [ ]:
#@title Remove empty rows
# #https://stackoverflow.com/questions/32093829/remove-duplicates-from-dataframe-based-on-two-columns-a-b-keeping-row-with-max
# # Drop rows with any duplicate cell and reset indext removing the original index
# df = df.drop_duplicates(['genre','prog_name']).reset_index(drop=True)
# #or
# df = df.drop_duplicates()
# #https://stackoverflow.com/questions/58311140/remove-white-space-from-entire-dataframe
# #https://hackersandslackers.com/pandas-dataframe-drop/#:~:text=Drop%20Empty%20Rows%20or%20Columns,method%20is%20specifically%20for%20this.&text=Technically%20you%20could%20run%20df,rows%20where%20are%20completely%20empty.
# # Drop rows with any empty or white space cell
for column in df.columns:
    df[column] = df[column].apply(lambda x:x.strip())
#https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python
#https://xspdf.com/resolution/56076054.html
#how the df looks like
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["genre"], inplace=True)
df.dropna(subset = ["prog_name"], inplace=True)

In [ ]:
#@title Create df only with import words
#https://stackoverflow.com/questions/3368969/find-string-between-two-substrings
#sort only import words


In [ ]:
#@title Create Genres dataframe and count it
#https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
# create datagrame for Genre
# df = pd.DataFrame(Prog_genr)
# df.columns =['prog_name','genre','desc']
print(len(df.index))
df

1347


,prog_name,genre,desc
0,系統資訊,類別：知性（知識）,系統資訊
1,數位小百科,類別：知性（知識）,數位小百科
2,頻道宣傳,類別：知性（知識）,頻道宣傳
3,系統資訊,類別：知性（知識）,系統資訊
4,數位小百科,類別：知性（知識）,數位小百科
...,...,...,...
1347,YOYO好好玩:1,類別：卡通/兒童、綜藝/娛樂,YOYO好好玩
1348,超級總動員第12季:2,類別：綜藝/娛樂、卡通/兒童,對於海綿寶寶和萌學園你了解多少呢？你是海綿寶寶達人嗎？還是你是萌學園達人？或者你是深藏不露的...
1349,超級總動員第12季:3,類別：卡通/兒童、綜藝/娛樂,對於海綿寶寶和萌學園你了解多少呢？你是海綿寶寶達人嗎？還是你是萌學園達人？或者你是深藏不露的...
1350,機甲戰龍第2季:14,類別：卡通/兒童,湯姆和他三位同學找到一個奇怪的棋盤遊戲。他們並不知道的是，每當他們擲一次骰子，就等於打開了一...
